In [1]:
import re

# Load the book's text
with open('data/TKMBFullBook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
chapters = re.split(r'Chapter \d+', text)
if chapters[0].strip() == "":
    chapters.pop(0) # Remove the empty string before the first chapter

In [ ]:
WINDOW = 80 # Number of words in each chunk
STRIDE = 20 # Number of words to shift for the next chunk

def chapter_to_chunks(chapter_text):
    # Simple word splitting (preserving original text for transformer tokenizer)
    words = chapter_text.split()
    chunks = []
    
    for start in range(0, len(words), STRIDE):
        end = start + WINDOW
        if end > len(words):
            # For the last chunk, take all remaining words
            chunk_words = words[start:]
        else:
            chunk_words = words[start:end]
        
        # Join the words back into text with single spaces
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
    
    return chunks

all_chunks = []
for chap_num, chap_text in enumerate(chapters, start=1):
    for i, chunk in enumerate(chapter_to_chunks(chap_text), start=1):
        all_chunks.append({
            "chapter": chap_num,
            "chunk_id": f"{chap_num}_{i}",
            "text": chunk
        })
        
print(f"Total chunks created: {len(all_chunks)}")
print(f"Example chunk: {all_chunks[0]}")

Total chunks created: 856530
Example chunk: {'chapter': 1, 'chunk_id': '1_1', 'text': '  \n C h a p t e r   1 \n   W h e n   h e   w a s   n e a r l y   t h i r t e e n ,   m y   b r o t h e r   J e m   g o t   h i s   a r m   b a d l y   b r o k e'}
